In [1]:
# Dependencies
import pandas as pd
from urllib.request import urlopen
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import requests

In [2]:
all_quotes = []

#Starting main for loop, that goes through each page
for i in range(1, 11):
    url = f'https://quotes.toscrape.com/page/{i}/'
    page = urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    quotes = soup.find_all('div', class_='quote')
    
    #Setting up page number and quote number
    page_number = 0
    number_of_quotes_per_page = 10
    quote_id = (i - 1)  * number_of_quotes_per_page
    

    #Starting the for loop
    for quote in quotes:
        #Quote number
        quote_id = quote_id + 1
        quote_info = quote_id
        

        
        #Getting the quote and author name
        quoteText  = quote.find('span', class_='text').text
        author = quote.find('small', class_='author').text
        
        #Getting the author url
        q_url = quote.a['href']
        author_url = urljoin( url ,  q_url)
        author_link = []
        author_link.append(author_url)
        
        
        #Getting all the tags for each quote
        tags = quote.find('div', class_='tags').find_all('a')
        tags_list = []
        for tag in tags:
            tags_list.append(tag.text)
        
        #Creating a dictionary for MongoDB
        data = {
            '_id' : quote_info,
            'quote' : quoteText,
            'name' : author,
            'tags' : tags_list,
            'author_url' : author_url
        }

        all_quotes.append(data)

In [9]:
all_quotes[0]

{'_id': 1,
 'quote': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 'name': 'Albert Einstein',
 'tags': ['change', 'deep-thoughts', 'thinking', 'world'],
 'author_url': 'https://quotes.toscrape.com/author/Albert-Einstein'}

In [24]:
#Getting list of author urls
a_url = [a['author_url'] for a in all_quotes]
# a_url

In [28]:
author_list = []
for link in a_url:
    response_2 = requests.get(link)
    print(response_2.text)
    for res in response_2:
        #Getting the quote and author name
        des  = res.find('div', class_='author-description').text
    #     author = link.find('h3', class_='author-title')

        #Creating a dictionary for MongoDB
        author_info = {
            'des' : des
            }

        author_list.append(author_info)



<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="UTF-8">
	<title>Quotes to Scrape</title>
    <link rel="stylesheet" href="/static/bootstrap.min.css">
    <link rel="stylesheet" href="/static/main.css">
</head>
<body>
    <div class="container">
        <div class="row header-box">
            <div class="col-md-8">
                <h1>
                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>
                </h1>
            </div>
            <div class="col-md-4">
                <p>
                
                    <a href="/login">Login</a>
                
                </p>
            </div>
        </div>
    

<div class="author-details">
    <h3 class="author-title">Albert Einstein</h2>
    <p><strong>Born:</strong> <span class="author-born-date">March 14, 1879</span> <span class="author-born-location">in Ulm, Germany</span></p>
    <p><strong>Description:</strong></p>
    <div class="author-description">
        In 1879, Albert Einstei

TypeError: find() takes no keyword arguments

In [19]:
author_list[0]

IndexError: list index out of range

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.test_quote_db
collection = db.quote1

# Inserting all the data we scraped
collection.insert_many(all_quotes)

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.test_quote_db
collection = db.author1

# Inserting all the data we scraped
collection.insert_many(author_list)